<a href="https://colab.research.google.com/github/KrishanuSinha/Krishanu_Repo/blob/master/%E2%9C%89%EF%B8%8F_MarketMail_AI_%E2%9C%89%EF%B8%8F_Fine_tuning_BLOOM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune a BLOOM-based ad generation model using `peft`, `transformers` and `bitsandbytes`

We can use the [MarketMail-AI dataset](https://huggingface.co/datasets/FourthBrainGenAI/MarketMail-AI) to fine-tune BLOOM to be able to generate marketing emails based off of a product and its description!

### Overview of PEFT and LoRA:

Based on some awesome new research [here](https://github.com/huggingface/peft), we can leverage techniques like PEFT and LoRA to train/fine-tune large models a lot more efficiently.

It can't be explained much better than the overview given in the above link:

```
Parameter-Efficient Fine-Tuning (PEFT) methods enable efficient adaptation of
pre-trained language models (PLMs) to various downstream applications without
fine-tuning all the model's parameters. Fine-tuning large-scale PLMs is often
prohibitively costly. In this regard, PEFT methods only fine-tune a small
number of (extra) model parameters, thereby greatly decreasing the
computational and storage costs. Recent State-of-the-Art PEFT techniques
achieve performance comparable to that of full fine-tuning.
```

### Install requirements

First, run the cells below to install the requirements:

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q -U git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9
!pip install -q git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Model loading

Let's load the `bloom-1b7` model!

We're also going to load the `bigscience/tokenizer` which is the tokenizer for all of the BLOOM models.

This step will take some time, as we have to download the model weights which are ~3.44GB.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7",
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/tokenizer")

### Post-processing on the model

Finally, we need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast the layer-norm in `float32` for stability. We also cast the output of the last layer in `float32` for the same reasons.

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Apply LoRA

Here comes the magic with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3145728 || all params: 1725554688 || trainable%: 0.18230242262828822


### Preprocessing

We can simply load our dataset from 🤗 Hugging Face with the `load_dataset` method!

In [ ]:
import transformers
from datasets import load_dataset

dataset_name = "kuokxuen/marketing_dataset"
product_name = "product"
product_desc = "description"
product_ad = "marketing_email"

In [ ]:
dataset = load_dataset(dataset_name)
print(dataset)
print(dataset['train'][0])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['product', 'description', 'marketing_email'],
        num_rows: 100
    })
})
{'product': 'LumoLife', 'description': 'A smart wearable device that tracks your posture and reminds you to sit up straight', 'marketing_email': "Subject: Say goodbye to slouching with LumoLife!\n\nHey there,\n\nAre you tired of slouching and feeling like a hunched-over hermit? Well, fret no more because LumoLife is here to save the day!\n\nIntroducing LumoLife - the smartest wearable device your back has been waiting for! This fantastic invention tracks your posture and gives sly reminders to straighten up and fly right. Say goodbye to those pesky slouching habits and hello to the confident, upright position you deserve.\n\nWhether you're sitting at your desk, slaying it in the boardroom, or even enjoying a mocha latte at your favorite cafe - LumoLife has got your back. Literally! With its sleek and lightweight design, it seamlessly blends into your every

In [ ]:
print(dataset['train'][0:10])

{'product': ['LumoLife', 'ChillZone', 'FlexiGrip', 'GlowPod', 'SleepScent', 'SnapCharge', 'FoodKeeper', 'SoundScenes', 'SwiftBlend', 'CleanSweep'], 'description': ['A smart wearable device that tracks your posture and reminds you to sit up straight', 'A compact portable cooler with built-in speakers, perfect for beach days and picnics', 'A phone case with an adjustable grip, providing better comfort and preventing accidental drops', 'An LED-lit plant pot that enhances indoor gardening and adds ambient lighting to any space', 'A pillowcase infused with relaxing aromatherapy scents, promoting better sleep', 'A smartphone charger with a retractable cable, eliminating tangled cords and clutter', 'A smart food storage container that detects freshness and sends notifications to your phone', 'Wireless headphones with customizable sound profiles for an immersive audio experience', 'A portable blender that doubles as a water bottle, perfect for on-the-go smoothies', 'An automatic robot mop that

We want to put our data in the form:

```
Below is a product and description, please write a marketing email for this product.

### Product
PRODUCT NAME

### Description:
DESCRIPTION

### Marketing Email:
OUR EMAIL HERE
```

This way, we can prompt our model well and receive the responses we want!

This is what fine-tuning, and prompt-engineering, is really all about!

In [ ]:
def generate_prompt(product: str, description: str, marketing_email: str) -> str:
  prompt = f"Below is a product and description, please write a marketing email for this product.\n\n### Product:\n{product}\n### Description:\n{description}\n\n### Marketing Email:\n{marketing_email}"
  return prompt

mapped_dataset = dataset.map(lambda samples: tokenizer(generate_prompt(samples['product'], samples['description'], samples['marketing_email'])))

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=mapped_dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=100,
        learning_rate=1e-3,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,2.421600
2,2.472300
3,2.365800
4,2.452700
5,2.467900
6,2.425100
7,2.403600
8,2.369700
9,2.398400
10,2.364000


TrainOutput(global_step=100, training_loss=1.5336379492282868, metrics={'train_runtime': 1605.0154, 'train_samples_per_second': 0.997, 'train_steps_per_second': 0.062, 'total_flos': 6286290573656064.0, 'train_loss': 1.5336379492282868, 'epoch': 16.0})

## Share adapters on the 🤗 Hub

In [ ]:
HUGGING_FACE_USER_NAME = "krishanusinha20"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_name = "marketmail"

model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}", use_auth_token=True)

adapter_model.bin:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/krishanusinha20/marketmail/commit/b69d685e54b8c7027b34500a3b827524ed1f0ee7', commit_message='Upload model', commit_description='', oid='b69d685e54b8c7027b34500a3b827524ed1f0ee7', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

You can also directly load adapters from the Hub using the commands below:

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

HUGGING_FACE_USER_NAME = "krishanusinha20"
model_name = "marketmail"

peft_model_id = f"{HUGGING_FACE_USER_NAME}/{model_name}"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

## Inference

You can then directly use the trained model or the model that you have loaded from the 🤗 Hub for inference as you would do it usually in `transformers`.

### Take it for a spin!

In [ ]:
from IPython.display import display, Markdown

def make_inference(product, description):
  batch = tokenizer(f"Below is a product and description, please write a marketing email for this product.\n\n### Product:\n{product}\n### Description:\n{description}\n\n### Marketing Email:\n", return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=200)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

In [ ]:
your_product_name_here = "The Coolinator"
your_product_description_here = "A personal cooling device to keep you from getting overheated on a hot summer's day!"

make_inference(your_product_name_here, your_product_description_here)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1448: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Below is a product and description, please write a marketing email for this product.

### Product:
The Coolinator
### Description:
A personal cooling device to keep you from getting overheated on a hot summer's day!

### Marketing Email:
Subject: Stay Cool with Coolinator - The Personal Cooling Device of Your Dreams!

Hey there,

Are you tired of sweaty days, overheated palms, and burning toasty coals burning brightly in the sun? Well, fret no more because we have the perfect solution for you - The Coolinator!

The Coolinator is the ultimate personal cooling device designed to keep you cool and comfortable on those scorching summer days. It is a compact, portable device that combines the convenience of a water bottle with the convenience of a personal cooling device. It's like having a personal cooling expert right at your fingertips!

The Coolinator is designed with you in mind. It's compact, portable, and easy to carry around, ensuring that you can keep you cool even when you're on the go. No more running around in a heat wave trying to find a cooling place or a place to sit down and relax. With The Coolinator, you can stay cool, stay

### Example in Training Set

In [ ]:
make_inference("SmartEyes", "Glasses with real-time translation")

Below is a product and description, please write a marketing email for this product.

### Product:
SmartEyes
### Description:
Glasses with real-time translation

### Marketing Email:
Subject: Elevate Your Style with SmartEyes Glasses! 🌟

Hey there fashionista,

Are you ready to take your style game to the next level? Introducing SmartEyes, the ultimate glasses that will transform your look into a fashion powerhouse!

Imagine having the power to speak multiple languages with these eyewear innovations. With real-time translation, you can effortlessly communicate your style preferences with no foreign languages or jargon to worry about. 🙌

Why SmartEyes are your new fashion ally:

1️⃣ Translator in the palm of your hand: Say hello to stylish communication!

2️⃣ Language of your choice: Say hello to endless conversation!

3️⃣ Powerful technology: Stay ahead of the trends and stay fashionable!

But that's not all, folks! SmartEyes is not just a fashion game-changer; it's also a health and wellness staple. Our innovative glasses are designed to keep you comfortable,

In [ ]:
make_inference("SmartInsuarance", "AI Auto Insuarance for quick auto insuarance quotes")

Below is a product and description, please write a marketing email for this product.

### Product:
SmartInsuarance
### Description:
AI Auto Insuarance for quick auto insuarance quotes

### Marketing Email:
Subject: Quote Me Smart, Say Goodbye to Insuarance Hangover!

Hey there,

Are you tired of waiting for the perfect auto insuarance quote to pop up in the blue sky? Well, we have the perfect solution for you! Introducing SmartInsuarance - the ultimate AI auto insuarance app!

With SmartInsuarance, you can say goodbye to insuarance hangover. This cutting-edge app allows you to quickly compare quotes from multiple car insurance companies, ensuring you get the best possible deal on your car insurance policy.

Why settle for the average when you can have the best? Here's why SmartInsuarance is the game-changer you never knew you needed:

1. Quick and easy: No more scrolling through endless websites or filling out long forms. Just select your insurance company, submit a quote, and get insuarance alerts in your inbox!

2. No more hassle or stress: Our platform is designed to

In [ ]:
make_inference("Yandy's Lingerie Collection", "Catalog of sexy lingerie caters to every desire, style and body type")

Below is a product and description, please write a marketing email for this product.

### Product:
Yandy's Lingerie Collection
### Description:
Catalog of sexy lingerie caters to every desire, style and body type

### Marketing Email:
Subject: Lingerie Caters to Every Dream, Every Style!

Hey there,

Are you ready to take your sexiness to the next level? Meet Yandy, the ultimate catalog of sexy lingerie! 💦🎉

We got you covered, babe! Introducing Yandy's Lingerie Collection, the ultimate game-changer for every wild, daring, and fabulous part of your body. 💦🎉

Say goodbye to your underwear drawer being too small for anything but the sexiest lingerie. With Yandy's Lingerie, you can be anything you want it to be. 💦🎉

Why settle for ordinary when you can be extraordinary? Here's why Yandy's Lingerie is the ultimate game-changer for every wild, daring, and fabulous part of your body:

1️⃣ Sexiest Collection: Get ready to wow your partner with the latest trends and hottest